# connecting notes to transfusions

In [20]:
import os
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
import numpy as np
pd.set_option('display.max_rows', 100)    # Show all rows



load_dotenv()
query_path = os.environ.get('BASE_QUERY_PATH')
client = bigquery.Client(os.environ.get('BIGQUERY_PROJECT_NAME'))

In [9]:
with open(f"{query_path}/completed_rbc_inputs.sql", 'r') as file:
    rbc_inputs_query = file.read()

rbc_inputs = client.query(rbc_inputs_query).to_dataframe()
rbc_inputs

/home/kmorhun/.pyenv/versions/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,subject_id,hadm_id,icustay_id,starttime,endtime,duration_minutes,itemid,rate,rateuom,label,amount,amountuom,totalamount,totalamountuom,statusdescription,cancelreason,isopenbag,orderid,linkorderid
0,55669,186586,275541,2127-06-23 13:00:00,2127-06-23 13:01:00,1,226368,NaN,None,OR Packed RBC Intake,281.000000,ml,281.0,ml,FinishedRunning,0,0,2751373,2751373
1,76058,103209,269471,2175-07-31 13:52:00,2175-07-31 13:53:00,1,226368,NaN,None,OR Packed RBC Intake,838.000000,ml,838.0,ml,FinishedRunning,0,0,912036,912036
2,51976,168578,266438,2111-09-24 00:32:00,2111-09-24 00:33:00,1,226368,NaN,None,OR Packed RBC Intake,270.000000,ml,270.0,ml,FinishedRunning,0,0,9364841,9364841
3,73200,114292,240326,2126-06-23 18:15:00,2126-06-23 18:16:00,1,226368,NaN,None,OR Packed RBC Intake,556.000000,ml,556.0,ml,FinishedRunning,0,0,7776410,7776410
4,61344,182494,225866,2137-12-01 13:46:00,2137-12-01 13:47:00,1,226368,NaN,None,OR Packed RBC Intake,631.000000,ml,631.0,ml,FinishedRunning,0,0,9317850,9317850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23308,71194,119417,264425,2180-10-06 00:00:00,2180-10-06 01:00:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,FinishedRunning,0,0,5972210,5972210
23309,90756,183509,247962,2130-09-19 22:26:00,2130-09-19 23:26:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,FinishedRunning,0,0,664515,664515
23310,45092,103810,297388,2157-01-24 20:40:00,2157-01-24 21:40:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,FinishedRunning,0,0,4538263,4538263
23311,44346,140114,250021,2165-04-06 11:27:00,2165-04-06 12:27:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,FinishedRunning,0,0,8817435,8817435


In [10]:
rbc_inputs.linkorderid.value_counts()

linkorderid
5507681    6
5570883    5
7465413    5
3796746    4
8361996    4
          ..
4193726    1
270395     1
3251099    1
3470533    1
8618897    1
Name: count, Length: 22971, dtype: Int64

In [15]:
print(rbc_inputs.isopenbag.value_counts())
print(rbc_inputs.statusdescription.value_counts())
print(rbc_inputs[rbc_inputs.isopenbag == 1].statusdescription.value_counts())

isopenbag
0    23021
1      292
Name: count, dtype: Int64
statusdescription
FinishedRunning    22709
Changed              294
Stopped              248
Rewritten             35
Paused                25
Flushed                2
Name: count, dtype: int64
statusdescription
FinishedRunning    289
Stopped              3
Name: count, dtype: int64


In [11]:
rbc_inputs[rbc_inputs.linkorderid == 5507681]

,subject_id,hadm_id,icustay_id,starttime,endtime,duration_minutes,itemid,rate,rateuom,label,amount,amountuom,totalamount,totalamountuom,statusdescription,cancelreason,isopenbag,orderid,linkorderid
357,65824,142509,277723,2132-03-09 15:07:00,2132-03-09 16:02:00,55,225168,149.767440,mL/hour,Packed Red Blood Cells,137.286820,ml,375.0,ml,Rewritten,0,0,256961,5507681
2207,65824,142509,277723,2132-03-09 14:04:00,2132-03-09 15:07:00,63,225168,124.806204,mL/hour,Packed Red Blood Cells,131.046514,ml,375.0,ml,Rewritten,0,0,6392111,5507681
3187,65824,142509,277723,2132-03-09 14:35:00,2132-03-09 15:52:00,77,225168,151.179426,mL/hour,Packed Red Blood Cells,194.013597,ml,375.0,ml,FinishedRunning,0,0,1382658,5507681
3597,65824,142509,277723,2132-03-09 14:43:00,2132-03-09 16:21:00,98,225168,124.489788,mL/hour,Packed Red Blood Cells,203.333320,ml,375.0,ml,Rewritten,0,0,8523819,5507681
7580,65824,142509,277723,2132-03-09 13:41:00,2132-03-09 14:35:00,54,225168,125.170068,mL/hour,Packed Red Blood Cells,112.653061,ml,375.0,ml,Changed,0,0,7922977,5507681
7847,65824,142509,277723,2132-03-09 13:00:00,2132-03-09 13:41:00,41,225168,99.999996,mL/hour,Packed Red Blood Cells,68.333331,ml,375.0,ml,Changed,0,0,5507681,5507681


In [28]:
#  actually, what am I doing? I should be looking at the ones with cancelreason != 0
with open(f"{query_path}/rbc_inputs.sql", 'r') as file:
    rbc_inputs_query = file.read()

all_rbc_inputs = client.query(rbc_inputs_query).to_dataframe()
cancelled_rbc_inputs = all_rbc_inputs[all_rbc_inputs.cancelreason != 0]
cancelled_rbc_inputs

/home/kmorhun/.pyenv/versions/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,subject_id,hadm_id,icustay_id,starttime,endtime,duration_minutes,itemid,rate,rateuom,label,amount,amountuom,totalamount,totalamountuom,statusdescription,cancelreason,isopenbag,orderid,linkorderid
1646,81407,167539,266615,2141-02-22 23:51:00,2141-02-22 23:52:00,1,226368,NaN,None,OR Packed RBC Intake,5950.000000,ml,5950.0,ml,Rewritten,1,0,8894426,8894426
1647,71974,158924,227483,2185-01-11 10:04:00,2185-01-11 10:05:00,1,226368,NaN,None,OR Packed RBC Intake,650.000000,ml,650.0,ml,Rewritten,1,0,6878603,6878603
1648,67771,158329,246263,2160-12-06 02:02:00,2160-12-06 02:03:00,1,226368,NaN,None,OR Packed RBC Intake,541.000000,ml,541.0,ml,Rewritten,1,0,4200083,4200083
1649,63201,190733,247444,2200-08-31 10:50:00,2200-08-31 10:51:00,1,226368,NaN,None,OR Packed RBC Intake,275.000000,ml,275.0,ml,Rewritten,1,0,8869789,8869789
1650,63201,190733,247444,2200-08-31 11:08:00,2200-08-31 11:09:00,1,226368,NaN,None,OR Packed RBC Intake,275.000000,ml,275.0,ml,Rewritten,1,0,1475827,1475827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26879,70278,173005,278392,2149-03-04 01:30:00,2149-03-04 02:30:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,Rewritten,1,0,5854291,5854291
26880,82826,138541,235365,2150-09-16 04:17:00,2150-09-16 05:17:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,Rewritten,1,0,7930582,7930582
26910,43742,191527,295473,2175-12-08 13:47:00,2175-12-08 14:47:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,Rewritten,1,0,4222577,4222577
26911,31584,106339,267813,2142-09-29 02:54:00,2142-09-29 03:54:00,60,225168,283.000002,mL/hour,Packed Red Blood Cells,283.000002,ml,283.0,ml,Rewritten,1,0,2863570,2863570


In [22]:
print(cancelled_rbc_inputs.linkorderid.value_counts())

#how many transfusion events have multiple entries?
np.sum(cancelled_rbc_inputs.linkorderid.value_counts() > 1)

linkorderid
633553     4
9527831    3
8003089    3
2230570    3
185620     3
          ..
3809747    1
6406601    1
1793077    1
7822465    1
3156811    1
Name: count, Length: 3535, dtype: Int64


np.int64(58)

In [23]:
cancelled_rbc_inputs[cancelled_rbc_inputs.linkorderid == 633553]

,subject_id,hadm_id,icustay_id,starttime,endtime,duration_minutes,itemid,rate,rateuom,label,amount,amountuom,totalamount,totalamountuom,statusdescription,cancelreason,orderid,linkorderid
3595,98514,101117,269040,2121-09-10 21:05:00,2121-09-10 22:15:00,70,225168,150.406506,mL/hour,Packed Red Blood Cells,175.474257,ml,350.0,ml,Rewritten,1,6772062,633553
3617,98514,101117,269040,2121-09-10 22:15:00,2121-09-10 22:16:00,1,225168,3000.000000,mL/hour,Packed Red Blood Cells,50.000000,ml,350.0,ml,Rewritten,1,9353400,633553
9259,98514,101117,269040,2121-09-10 20:45:00,2121-09-10 21:05:00,20,225168,124.999998,mL/hour,Packed Red Blood Cells,41.666666,ml,350.0,ml,Rewritten,1,633553,633553
9421,98514,101117,269040,2121-09-10 22:16:00,2121-09-10 22:49:00,33,225168,150.652872,mL/hour,Packed Red Blood Cells,82.859080,ml,350.0,ml,Rewritten,1,8834852,633553


In [27]:
print(cancelled_rbc_inputs.isopenbag.value_counts())
print(cancelled_rbc_inputs.statusdescription.value_counts())
print(cancelled_rbc_inputs[cancelled_rbc_inputs.isopenbag == 1].statusdescription.value_counts())
# all status descriptions are rewritten.

isopenbag
0    3576
1      24
Name: count, dtype: Int64
statusdescription
Rewritten    3600
Name: count, dtype: int64
statusdescription
Rewritten    24
Name: count, dtype: int64


In [29]:
print(all_rbc_inputs.statusdescription.value_counts())

statusdescription
FinishedRunning    22709
Rewritten           3635
Changed              294
Stopped              248
Paused                25
Flushed                2
Name: count, dtype: int64


# Finding notes in a df given a property and a desired value

In [50]:
# with open(f"{query_path}/scratch.sql", 'r') as file:
#     test = file.read()

# test_df = client.query(test).to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/transfusion-reactions/queries/c0d354e2-1044-462e-9eaf-1e31b54cf2e4?maxResults=0&location=US&prettyPrint=false: Resources exceeded during query execution: Your project or organization exceeded the maximum disk and memory limit available for shuffle operations. Consider provisioning more slots, reducing query concurrency, or using more efficient logic in this job.. at [55:1]

Location: US
Job ID: c0d354e2-1044-462e-9eaf-1e31b54cf2e4


In [ ]:
def get_notes_from_df(table_query_filename):
    """
    returns a df containing orderid, linkorderid, and note properties, derived from a particular subset of inputevents.
    These notes will be the most recent note written after the latest end time of a set of inputevents with the same linkorderid

    In other words: 
    For each unique linkorderid:
        1. find the entry with the largest endtime
        2. find the first note with a chartdate that is after that endtime
        3. place that note into a table, associated with its inputevent via linkorderid
    Args: 
        table_query_filename: the sql file describing the table to pull note events from. e.g. rbc_inputs.sql
        property: a column label in the table to use as the anchor to limit note searches for
        value: the value of that column label in the table to use as the anchor to limit note searches for
    """
    with open(table_query_filename, 'r') as file:
        table_query = file.read()
    
    query = "CREATE TEMPORARY TABLE all_inputs AS\n"
    query += table_query + "\n"

    """-- Get notes with matching patients records 
    SELECT * 
    FROM physionet-data.mimiciii_notes.noteevents
    WHERE (SUBJECT_ID IN (SELECT subject_id FROM all_blood_inputs))
        -- Selects notes on start date or one day after end day 
        -- Note: dates are internally consistent in the database: https://mimic.mit.edu/docs/iii/about/time/
        AND (
            DATE(CHARTDATE) IN (SELECT DATE(starttime) FROM all_blood_inputs)
            OR DATE(CHARTDATE) IN (SELECT DATE(endtime + INTERVAL 1 DAY) FROM all_blood_inputs)
        );
    """

    

In [ ]:
# # Ensure datetime types for comparison
# all_rbc_inputs['endtime'] = pd.to_datetime(all_rbc_inputs['endtime'])
# noteevents['chartdate'] = pd.to_datetime(noteevents['chartdate'])

# # Step 1: Find the largest `endtime` for each `linkorderid`
# latest_endtime = all_rbc_inputs.groupby('linkorderid')['endtime'].max().reset_index()
# latest_endtime.rename(columns={'endtime': 'latest_endtime'}, inplace=True)

# # Step 2: Merge to get notes with `chartdate` > `latest_endtime`
# merged = pd.merge(noteevents, latest_endtime, on='hadm_id')

# # Step 3: Filter for notes after the latest_endtime
# filtered_notes = merged[merged['chartdate'] > merged['latest_endtime']]

# # Step 4: Find the first note for each `linkorderid` after the `latest_endtime`
# first_notes = (
#     filtered_notes.sort_values(by=['linkorderid', 'chartdate'])  # Sort by linkorderid and chartdate
#     .groupby('linkorderid')
#     .first()  # Take the first note for each group
#     .reset_index()
# )

# # Final result
# print(first_notes)